# Numerical Assignment

In [30]:
#!/usr/bin/env python3
#==============================================================================
# polytrope_shooting.py
#
# ***
#
# Author: Stanley A. Baronett
# Created: 2022-11-19
# Updated: 2022-11-19
#==============================================================================
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# (HKT04, eqs. 7.45, 7.48)
yfunc = lambda x, n : 1 - (1/6)*x**2 + (n/120)*x**4 - (n*(8*n - 5)/15120)*x**6
zfunc = lambda x, n : -(1/3)*x + (n/30)*x**3 - (n*(8*n - 5)/2520)*x**5
yp = lambda x, y, z, n : z
zp = lambda x, y, z, n : -y**n - (2/x)*z

def rk4(n, xi, yi, zi, h):
    """
    (HKT04, eq. 7.46)
    """
    k1 = h*yp(xi, yi, zi, n)
    l1 = h*zp(xi, yi, zi, n)
    k2 = h*yp(xi + h/2, yi + k1/2, zi + l1/2, n)
    l2 = h*zp(xi + h/2, yi + k1/2, zi + l1/2, n)
    k3 = h*yp(xi + h/2, yi + k2/2, zi + l2/2, n)
    l3 = h*zp(xi + h/2, yi + k2/2, zi + l2/2, n)
    k4 = h*yp(xi + h, yi + k3, zi + l3, n)
    l4 = h*zp(xi + h, yi + k3, zi + l3, n)
    xi1 = xi + h
    yi1 = yi + k1/6 + k2/3 + k3/3 + k4/6
    zi1 = zi + l1/6 + l2/3 + l3/3 + l4/6

    return xi1, yi1, zi1

# Start
ns = [0, 1.0, 1.5, 2.0, 3.0, 4.0] # polytrope indices

print('-----------------------------------------------')
print('Index n\t  ξ₁\t\t-θₙ′(ξ₁)\tρ_c/<ρ>')
print('-----------------------------------------------')

for n in ns:
    x = 1e-16                         # ξ
    y = yfunc(x, n)                   # θₙ
    z = zfunc(x, n)                   # dθₙ/dξ = dy/dx
    h = 1e-5                          # step size

    while y.real > 0:
        x, y, z = rk4(n, x, y, z, h)

    print(f'  {n:.1f}\t{x:.4f}\t\t{-z.real:.5f}\t')

-----------------------------------------------
Index n	  ξ₁		-θₙ′(ξ₁)	ρ_c/<ρ>
-----------------------------------------------
  0.0	2.4495		0.81650	
  1.0	3.1416		0.31831	
  1.5	3.6538		0.20330	
  2.0	4.3529		0.12725	
  3.0	6.8968		0.04243	
  4.0	14.9715		0.00802	


In [22]:
xref = np.sqrt(6)
ypref = np.sqrt(6)/3

print(f'  {n:.1f}\t{xref:.4f}\t\t{ypref:.4f}\t')

  4.0	2.4495		0.8165	
